In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras
import random
import numpy
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
!pip install gdown

In [ ]:
!gdown --id 1-7u4MzeuZ3OSnB4bn2lrGYhALxcCBSNn

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-7u4MzeuZ3OSnB4bn2lrGYhALxcCBSNn
From (redirected): https://drive.google.com/uc?id=1-7u4MzeuZ3OSnB4bn2lrGYhALxcCBSNn&confirm=t&uuid=f05a09c4-92db-4631-bf3d-ad616359b9b1
To: /content/train.zip
100% 1.88G/1.88G [00:13<00:00, 144MB/s]


In [ ]:
import zipfile
zip_path = '/content/train.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

sample_data  train  train.zip


In [ ]:
!head -20 /content/train/classes.txt

1 kannada
2 tamil
3 telugu
4 malayalam

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [ ]:


batch_size = 32
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        '/content/train',  # training images
        target_size=(200, 200),  # resized to 200 x 200
        batch_size=batch_size,
        classes = ['1','2','3','4'],
        class_mode='categorical')


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()



model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])

total_sample=train_generator.n

n_epochs = 30

history = model.fit_generator( train_generator, steps_per_epoch=int(total_sample/batch_size), epochs=n_epochs, verbose=1)

model.save('model.h5')



Found 37441 images belonging to 4 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2

<ipython-input-7-c9f5799fac57>:67: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
1170/1170 [==============================] - 132s 108ms/step - loss: 0.5047 - acc: 0.7819
Epoch 2/30
1170/1170 [==============================] - 127s 108ms/step - loss: 0.1476 - acc: 0.9456
Epoch 3/30
1170/1170 [==============================] - 129s 110ms/step - loss: 0.0868 - acc: 0.9693
Epoch 4/30
1170/1170 [==============================] - 131s 111ms/step - loss: 0.0588 - acc: 0.9788
Epoch 5/30
1170/1170 [==============================] - 130s 111ms/step - loss: 0.0457 - acc: 0.9848
Epoch 6/30
1170/1170 [==============================] - 126s 107ms/step - loss: 0.0430 - acc: 0.9870
Epoch 7/30
1170/1170 [==============================] - 123s 105ms/step - loss: 0.0324 - acc: 0.9900
Epoch 8/30
1170/1170 [==============================] - 123s 105ms/step - loss: 0.0320 - acc: 0.9907
Epoch 9/30
1170/1170 [==============================] - 125s 107ms/step - loss: 0.0290 - acc: 0.9919
Epoch 10/30
1170/1170 [==============================] - 124s 106ms/step - loss: 0.0265 - a

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('4.jpg', target_size = (200,200))
#test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)

if result[0][0] == 1:
    print("kn")
elif result[0][1] == 1:
    print("ta")
elif result[0][2] == 1:
    print("tl")
elif result[0][3] == 1:
    print("ml")



1/1 [==============================] - 0s 19ms/step
ta


In [ ]:
print(result)

[[0. 1. 0. 0.]]
